In [2]:
import json
import time
import numpy as np
import os
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import cv2
from pathlib import Path


# Getting File

In [5]:
# Opening JSON file
start = time.time()
with open('dict_spottings.json') as json_file:
    data = dict(json.load(json_file))

In [6]:
def removeType(string):
    return string[:-4]

In [7]:
videoLocation = 'D:/BOBSL/BOBSL200G/bobsl/videos'
videoTitles = os.listdir(videoLocation)

videoName = videoTitles[0]
targetVideo = videoLocation + '/' +videoName

videoTitlesEdited = list(map(removeType, videoTitles))

# Only Loading in files that I have Downloaded

In [8]:
wordsTrain = list(dict(data['train']).keys())

train = []
for word in wordsTrain:
    wordData = data['train'][word]
    for i in range(len(wordData['names'])):
        fileName = wordData['names'][i]
        if fileName in videoTitlesEdited:
            globalTime = wordData['global_times'][i]
            probs = wordData['probs'][i]
            train.append((word, fileName, globalTime, probs))

In [9]:
wordsVal = list(dict(data['val']).keys())

val = []
for word in wordsVal:
    wordData = data['val'][word]
    for i in range(len(wordData['names'])):
        fileName = wordData['names'][i]
        if fileName in videoTitlesEdited:
            globalTime = wordData['global_times'][i]
            probs = wordData['probs'][i]
            val.append((word, fileName, globalTime, probs))

In [10]:
wordsTest = list(dict(data['test']).keys())

test = []
for word in wordsTest:
    wordData = data['test'][word]
    for i in range(len(wordData['names'])):
        fileName = wordData['names'][i]
        if fileName in videoTitlesEdited:
            globalTime = wordData['global_times'][i]
            probs = wordData['probs'][i]
            test.append((word, fileName, globalTime, probs))

In [63]:
print(len(train))
print(len(val))
print(len(test))

2758672
112184
414790


## Create new file showing information about files i have

In [65]:
f1 = open("train.txt", "w")
for line in train:
    f1.write(str(line) + '\n')
f1.close()

f2 = open("val.txt", "w")
for line in val:
    f2.write(str(line) + '\n')
f2.close()

f3 = open("test.txt", "w")
for line in test:
    f3.write(str(line) + '\n')
f3.close()

# Creating smaller videos

In [11]:
sortedVideoList = []
for videoName in videoTitlesEdited:
    targeVideoInfo = []
    for wordInfo in val: # wordInfo = (word, fileName, globalTime, probs)
        if wordInfo[1] == videoName:
            targeVideoInfo.append(wordInfo)
    if len(targeVideoInfo) > 0:  
        sortedVideoList.append(targeVideoInfo)
sortedVideoList

[[('1', '5213407827313563421', 63.2, 0.79419),
  ('1', '5213407827313563421', 188.8, 0.82758),
  ('1', '5213407827313563421', 457.76, 0.8011),
  ('1', '5213407827313563421', 457.76, 0.8011),
  ('1', '5213407827313563421', 477.76, 0.72512),
  ('1', '5213407827313563421', 516.32, 0.71467),
  ('1', '5213407827313563421', 799.52, 0.72857),
  ('1', '5213407827313563421', 1051.04, 0.73824),
  ('1', '5213407827313563421', 1123.52, 0.74813),
  ('1', '5213407827313563421', 1351.68, 0.74029),
  ('1', '5213407827313563421', 1351.68, 0.74029),
  ('1', '5213407827313563421', 1423.68, 0.74991),
  ('1', '5213407827313563421', 1423.68, 0.74991),
  ('1', '5213407827313563421', 1437.76, 0.71505),
  ('1', '5213407827313563421', 1453.76, 0.72748),
  ('100', '5213407827313563421', 1049.76, 0.75347),
  ('1st', '5213407827313563421', 215.52, 0.7307),
  ('1st', '5213407827313563421', 765.28, 0.70436),
  ('2', '5213407827313563421', 162.56, 0.81283),
  ('2', '5213407827313563421', 187.84, 0.86731),
  ('2', '52

In [41]:
video1 = sortedVideoList[0]
FPS = 25
parts = []
for info in video1:
    globalTime = info[2]
    frameStart = globalTime
    frameEnd = globalTime + 1.5
    parts.append((frameStart, frameEnd, info[0]))
parts = sorted(parts, key=lambda x: x[0])
f4 = open("check.txt", "w")
for line in parts:
    f4.write(str(line) + '\n')
f4.close()

('after', '5213407827313563421', 907.52, 0.75421)

In [21]:

videoLocation = 'D:/BOBSL/BOBSL200G/bobsl/videos'
videoTitles = os.listdir(videoLocation)

saveLocation = 'D:/Thesis/Video-Segments'

videoName = video1[0][1]

In [14]:
def extract_segment(input_file, start_time, end_time, output_file):
    cap = cv2.VideoCapture(input_file)

    # Get video properties
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Calculate start and end frame
    start_frame = int(start_time * fps)
    end_frame = int(end_time * fps)
    print(fps)
    print(start_frame)
    print(end_frame)

    # Set video writer
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_file, fourcc, fps, (width, height))

    # Extract frames and write to output file
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame_num = int(cap.get(cv2.CAP_PROP_POS_FRAMES))
        if frame_num < start_frame:
            continue
        if frame_num > end_frame:
            break

        out.write(frame)

    # Release resources
    cap.release()
    out.release()

In [20]:
videoName = video1[0][1]
targetVideo = videoLocation + '/' + videoName + '.mp4'
saveLocation = 'D:/Thesis/Video-Segments'
j = 0
for start, end, word in parts:
    if j == 50:
        break
    
    output  = saveLocation + '/' + videoName + '-' + word + '-'+ str(int(start)) +'-' + str(int(end)) + '.mp4'
    path = Path(output)
    i = 1
    while(path.is_file()):
        output = saveLocation + '/' + videoName + '-' + word + '-'+ str(int(start)) +'-' + str(int(end)) + str(i) + '.mp4'
        path = Path(output)
        i += 1
    print(output, start, end, word)
    extract_segment(targetVideo, start, end, output)
    j += 1


D:/Thesis/Video-Segments/5213407827313563421-to-0-1.mp4 0.16 1.66 to
25.0
4
41
D:/Thesis/Video-Segments/5213407827313563421-bcc-3-5.mp4 3.68 5.18 bcc
25.0
92
129
D:/Thesis/Video-Segments/5213407827313563421-south-africa-4-5.mp4 4.16 5.66 south-africa
25.0
104
141
D:/Thesis/Video-Segments/5213407827313563421-blow-4-6.mp4 4.8 6.3 blow
25.0
120
157
D:/Thesis/Video-Segments/5213407827313563421-special-5-7.mp4 5.6 7.1 special
25.0
140
177
D:/Thesis/Video-Segments/5213407827313563421-all-5-7.mp4 5.92 7.42 all
25.0
148
185
D:/Thesis/Video-Segments/5213407827313563421-got-6-7.mp4 6.08 7.58 got
25.0
152
189
D:/Thesis/Video-Segments/5213407827313563421-xor-6-8.mp4 6.88 8.379999999999999 xor
25.0
172
209
D:/Thesis/Video-Segments/5213407827313563421-now-7-8.mp4 7.04 8.54 now
25.0
176
213
D:/Thesis/Video-Segments/5213407827313563421-south-7-8.mp4 7.04 8.54 south
25.0
176
213
D:/Thesis/Video-Segments/5213407827313563421-africa-7-9.mp4 7.52 9.02 africa
25.0
188
225
D:/Thesis/Video-Segments/5213407827